# RL - Exercício 1 - Q-Learning Cliff Walking

## Trabalho desenvolvido durante o MBA em Data Science / IA na FIAP - 2022~2023.

## Prof. Felipe Teodoro

## Agente de Reinforcement Learning jogando Cliff Walking usando Q-Learning

Adaptado HuggingFace

<img src="https://www.gymlibrary.dev/_images/cliff_walking.gif" alt="Environments"/>

### 🎮 Environments:

- [CliffWalking-v0](https://www.gymlibrary.dev/environments/toy_text/cliff_walking/)


### 📚 RL-Library:

- Python and NumPy
- [Gym](https://www.gymlibrary.dev/)

## Instalar dependências e criar um display virtual 🔽


In [1]:
!pip install gym==0.24
!pip install pygame
!pip install numpy

!pip install huggingface_hub
!pip install pickle5
!pip install pyyaml==6.0
!pip install imageio
!pip install imageio_ffmpeg
!pip install pyglet==1.5.1
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/694.4 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━ 286.7/694.4 kB 7.5 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 686.1/694.4 kB 10.3 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 694.4/694.4 kB 9.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.24.0-py3-none-any.whl size=790674 sha256=a9ad0b5f31eb4e0ff257f3d68a5df7642e7e776f145a70f0cd8d53aff04af491
  Stored in directory: /root/.cache/pip/wheels/2f/a1/b1/5f4110c8943b36e6cdfcc0e5768481639fa042b3c8357e82cc
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.

In [2]:
%%capture
!apt update
!apt install ffmpeg xvfb
!pip install xvfbwrapper
!pip install pyvirtualdisplay

Para garantir que as novas bibliotecas instaladas sejam usadas, **às vezes é necessário reiniciar o tempo de execução do notebook**. A próxima célula forçará o **tempo de execução a travar, então você precisará se conectar novamente e executar o código a partir daqui**.

In [3]:
#import os
#os.kill(os.getpid(), 9)

In [4]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

## Importação de pacotes 📦

Além das bibliotecas instaladas, utilizamos também:

- `random`: Para gerar números aleatórios (que serão úteis para a política epsilon-greedy).
- `imageio`: Para gerar um vídeo de replay.

In [5]:
import numpy as np
import gym
import random
import imageio
import os
import tqdm
import time

import pickle5 as pickle
from tqdm.notebook import tqdm

# Part 1: CliffWalking

## Criando o ambiente CliffWalking-v0 (https://www.gymlibrary.dev/environments/toy_text/cliff_walking/)
---

💡 Um bom hábito quando você começa a usar um ambiente é verificar sua documentação

👉 https://www.gymlibrary.dev/environments/toy_text/cliff_walking/

---


In [6]:
env = gym.make("CliffWalking-v0")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Verifique o Environment:


In [7]:
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space", env.observation_space)
print("Sample observation", env.observation_space.sample()) # Get a random observation

_____OBSERVATION SPACE_____ 

Observation Space Discrete(48)
Sample observation 11


In [8]:
print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample()) # Take a random action


 _____ACTION SPACE_____ 

Action Space Shape 4
Action Space Sample 1


O espaço de ação - conjunto de ações possíveis que o agente pode realizar - é discreto com 4 ações disponíveis 🎮:
- 0: para cima
- 1: para direita
- 2: para baixo
- 3: para esquerda

Função de recompensa 💰:
- Atingir objetivo: +1
- Andar sem cair no penhasco: -1
- Cair no penhasco: -100

In [9]:
state_space = env.observation_space.n
print("There are ", state_space, " possible states")

action_space = env.action_space.n
print("There are ", action_space, " possible actions")

There are  48  possible states
There are  4  possible actions


In [10]:
# Vamos criar nossa Qtable de tamanho (state_space, action_space) e inicializar cada valor em 0 usando np.zeros
def initialize_q_table(state_space, action_space):
  Qtable = np.zeros((state_space, action_space))
  return Qtable

In [11]:
Qtable_cliffwalking = initialize_q_table(state_space, action_space)

In [12]:
Qtable_cliffwalking

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],


In [13]:
def greedy_policy(Qtable, state):
  # Exploitation: take the action with the highest state, action value
  action = np.argmax(Qtable[state][:])

  return action

In [14]:
def epsilon_greedy_policy(Qtable, state, epsilon):
  # Gera aleatoriamente um número entre 0 e 1
  random_int = random.uniform(0,1)
  # if random_int > maior que epsilon --> exploitation
  if random_int > epsilon:
     # Execute a ação com o maior valor dado um estado
     # np.argmax pode ser útil aqui
    action = greedy_policy(Qtable, state)
  # else --> exploration
  else:
    action = env.action_space.sample()

  return action

## Definindo os hiperparâmetros ⚙️
Os hiperparâmetros relacionados à exploração são alguns dos mais importantes.

- Precisamos garantir que nosso agente **explore o espaço de estados** o suficiente para aprender uma boa aproximação de valor. Para fazer isso, precisamos ter decaimento progressivo do epsilon.
- Se você diminuir o epsilon muito rápido (decay_rate muito alto), **você corre o risco de que seu agente fique preso**, já que seu agente não explorou o espaço de estado o suficiente e, portanto, não pode resolver o problema.

In [15]:
# Parâmetros de treinamento
n_training_episodes = 10000 # Total de episódios de treinamento
''' Anotação da aula
- um episódio de treinamento é uma "rodada" completa, uma
  tentativa de chegar no prêmio. Termina quando o agente cai penhasco,
  ou quando chega no objetivo.
'''
learning_rate = 0.7 # Taxa de aprendizado
''' Anotação da aula
- taxa de aprendizado não é fácil de calibrar e não tem método. Por convenção
  coloca-se algo entre 0.7 e 0.9
'''

# Parâmetros de avaliação
n_eval_episodes = 100 # Número total de episódios de teste

# Parâmetros do ambiente
env_id = "CliffWalking-v0" # Nome do ambiente
max_steps = 99 # Max passos por episódio
''' Anotação da aula
- Evita o agente ficar em loop infinito. Se em 99 passos, não acabar,
  encerra o episódio
'''
gamma = 0.95 # Taxa de desconto
eval_seed = [] # A semente de avaliação do ambiente

# Parâmetros de exploração
max_epsilon = 1.0 # Probabilidade de exploração no início
min_epsilon = 0.05 # Probabilidade mínima de exploração
decay_rate = 0.0001 # Taxa de decaimento exponencial para prob de exploração

## Rotina de Treinamento

In [16]:
def train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable):
  for episode in tqdm(range(n_training_episodes)):
    # # Reduzir epsilon (porque precisamos cada vez menos exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
    # Redefinir o ambiente
    state = env.reset()
    step = 0
    done = False

    # repete
    for step in range(max_steps):
      # Escolha a ação At para usar a política gananciosa (greedy policy) do epsilon
      action = epsilon_greedy_policy(Qtable, state, epsilon)

      # Take action At and observe Rt+1 and St+1
      # Take the action (a) and observe the outcome state(s') and reward (r)
      new_state, reward, done, info = env.step(action)

      # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
      Qtable[state][action] = Qtable[state][action] + learning_rate * (reward + gamma * np.max(Qtable[new_state]) - Qtable[state][action])

      # If done, finish the episode
      if done:
        break

      # Our next state is the new state
      state = new_state
  return Qtable

## Treinando o agente Q-Learning 🏃

In [17]:
Qtable_cliffwalking = train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable_cliffwalking)

  0%|          | 0/10000 [00:00<?, ?it/s]

## Verificando a tabela Q-Learning 👀

In [18]:
Qtable_cliffwalking

array([[ -10.7341754 ,  -10.24650042,  -10.24650042,  -10.7341754 ],
       [ -10.24650042,   -9.73315833,   -9.73315833,  -10.7341754 ],
       [  -9.73315833,   -9.19279825,   -9.19279825,  -10.24650042],
       [  -9.19279825,   -8.62399815,   -8.62399815,   -9.73315833],
       [  -8.62399815,   -8.02526122,   -8.02526122,   -9.19279825],
       [  -8.02526122,   -7.39501181,   -7.39501181,   -8.62399815],
       [  -7.39501181,   -6.73159137,   -6.73159137,   -8.02526122],
       [  -6.73159137,   -6.03325408,   -6.03325408,   -7.39501181],
       [  -6.03325408,   -5.29816219,   -5.29816219,   -6.73159137],
       [  -5.29816219,   -4.52438125,   -4.52438125,   -6.03325408],
       [  -4.52438125,   -3.709875  ,   -3.709875  ,   -5.29816219],
       [  -3.709875  ,   -3.709875  ,   -2.8525    ,   -4.52438125],
       [ -10.7341754 ,   -9.73315833,   -9.73315833,  -10.24650042],
       [ -10.24650042,   -9.19279825,   -9.19279825,  -10.24650042],
       [  -9.73315833,   -8.623998

## Avaliação do Método 📝

- Definimos o método de avaliação que vamos usar para testar nosso agente Q-Learning.

In [19]:
def evaluate_agent(env, max_steps, n_eval_episodes, Q, seed):
  """
   Avalie o agente para episódios ``n_eval_episodes`` e retorne recompensa média e padrão de recompensa.
   :param env: O ambiente de avaliação
   :param n_eval_episodes: Número de episódios para avaliar o agente
   :param Q: A tabela Q
   :param seed: A matriz de sementes de avaliação (para taxi-v3)
   """
  episode_rewards = []
  for episode in tqdm(range(n_eval_episodes)):
    if seed:
      state = env.reset(seed=seed[episode])
    else:
      state = env.reset()
    step = 0
    done = False
    total_rewards_ep = 0

    for step in range(max_steps):
      # Tome a ação (índice) que tem a recompensa futura máxima esperada dado aquele estado
      action = greedy_policy(Q, state)
      new_state, reward, done, info = env.step(action)
      total_rewards_ep += reward

      if done:
        break
      state = new_state
    episode_rewards.append(total_rewards_ep)
  mean_reward = np.mean(episode_rewards)
  std_reward = np.std(episode_rewards)

  return mean_reward, std_reward

## Avaliando nosso agente Q-Learning 📈


In [20]:
# Evaluate our Agent
mean_reward, std_reward = evaluate_agent(env, max_steps, n_eval_episodes, Qtable_cliffwalking, eval_seed)
print(f"Mean_reward={mean_reward:.2f} +/- {std_reward:.2f}")

  0%|          | 0/100 [00:00<?, ?it/s]

Mean_reward=-13.00 +/- 0.00


#### Não modifique essa parte


In [21]:
def play_actions(env, Qtable, delay = 1):
  """
   Gerar um vídeo de replay do agente
   :param env
   :param Qtable: Qtable do nosso agente
   :param out_directory
   :param fps: quantos quadros por segundo (com taxi-v3 e frozenlake-v1 usamos 1)
   """

  sequences = []
  done = False
  state = env.reset(seed=random.randint(0,500))
  txt = env.render(mode='human')

  sequences.append(txt)
  while not done:
    # Tome a ação (índice) que tem a recompensa futura máxima esperada dado aquele estado

    action = np.argmax(Qtable[state][:])
    print(action)
    state, reward, done, info = env.step(action) # Colocamos diretamente next_state = state para a lógica de gravação
    txt = env.render(mode='human')
    sequences.append(txt)
    print(txt)
    time.sleep(delay)

  return sequences


In [22]:
play_actions(env, Qtable_cliffwalking,  1)

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T

0
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

None
1
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  x  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

None
1
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  x  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

None
1
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  x  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

None
1
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  x  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

None
1
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  x  o  o  o  o  o  o
o  C  C  C  

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]